In [4]:
from repliclust import Archetype, DataGenerator, set_seed
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_mutual_info_score as ami
import numpy as np
from itertools import product

PCT = 0.025

def carry_out_benchmark(X, y, archetype):
    gauss = GaussianMixture(n_components=archetype.n_clusters,
                            max_iter=500, n_init=10,
                            init_params="kmeans")
    kmeans = KMeans(n_clusters=archetype.n_clusters,
                    max_iter=500, n_init=10,
                    init="k-means++")

    y_hat_kmeans = kmeans.fit_predict(X)
    ami_kmeans = ami(y, y_hat_kmeans)
    y_hat_gauss = gauss.fit_predict(X)
    ami_gauss = ami(y, y_hat_gauss)

    return (ami_kmeans, ami_gauss)

In [6]:
### Archetypes for Figure 11

samples_per_cluster = 100 # vary this number on {100, 200, 1000} to reproduce paper

highdim_shallow = Archetype(dim=200, n_clusters=10, n_samples=10*samples_per_cluster, distributions=['normal'],
                            name='highdim_shallow')
expon_highoverlap = Archetype(dim=10, n_clusters=5, n_samples=5*samples_per_cluster, max_overlap=(1+PCT)*0.20, min_overlap=(1-PCT)*0.20,
                              distributions=['exponential'],
                              name='expon_highoverlap')
normal_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*samples_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10, distributions=['normal'],
                                  name='normal_highlyvariable')
nonnormal_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*samples_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10, distributions=['exponential'],
                                  name='nonnormal_highlyvariable')
normal_easy = Archetype(dim=10, n_clusters=7, n_samples=7*samples_per_cluster, radius_maxmin=1, imbalance_ratio=1,
                                  aspect_ref=1, aspect_maxmin=1, name='normal_easy', 
                                  distributions=['normal'])
heavytailed = Archetype(dim=10, n_clusters=5, n_samples=5*samples_per_cluster, distributions=[('standard_t', {'df': 2})],
                        name='heavytailed')

archetype_collection = [highdim_shallow, expon_highoverlap, normal_highlyvariable, normal_easy, heavytailed]

In [7]:
### Archetypes for Figure 12

data_pts_per_cluster = 1000 # vary this number on {100, 200, 1000} to reproduce paper

normal_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*data_pts_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10, distributions=['normal'],
                                  name='normal_highlyvariable')
nonnormal_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*data_pts_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10, distributions=['exponential'],
                                  name='nonnormal_highlyvariable')
heavytails_soft_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*data_pts_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10,
                                  distributions=[('standard_t', {'df': 4})],
                                  name='heavytails_soft_highlyvariable')
heavytails_med_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*data_pts_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10,
                                  distributions=[('standard_t', {'df': 3})],
                                  name='heavytails_med_highlyvariable')
heavytails_hard_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*data_pts_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10,
                                  distributions=[('standard_t', {'df': 2})],
                                  name='heavytails_hard_highlyvariable')
heavytails_hardhard_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*data_pts_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10,
                                  distributions=[('standard_t', {'df': 1.5})],
                                  name='heavytails_hardhard_highlyvariable')
heavytails_hardhardhard_highlyvariable = Archetype(dim=10, n_clusters=7, n_samples=7*data_pts_per_cluster, radius_maxmin=10, imbalance_ratio=10,
                                  aspect_ref=3, aspect_maxmin=10,
                                  distributions=[('standard_t', {'df': 1.0})],
                                  name='heavytails_hardhardhard_highlyvariable')
highlyvariable_archetypes = [normal_highlyvariable,
                             nonnormal_highlyvariable,
                             heavytails_med_highlyvariable,
                             heavytails_hard_highlyvariable,
                             heavytails_hardhard_highlyvariable
                             ]

In [9]:
### Carry out the benchmark(s)

N_REPL = 300
collection = archetype_collection # change to highlyvariable_archetypes for second study
n_archetypes = len(collection)
dg = DataGenerator(collection)

data = []

for X, y, archetype in dg(n_datasets=N_REPL*n_archetypes, quiet=False):
    # carry out benchmark
    kmeans_result, gauss_result = carry_out_benchmark(X,y,archetype)

    # store the results
    data.append((kmeans_result, gauss_result, archetype.name))

Optimizing Cluster Centers: 100%|██████████| 300/300 [00:00<00:00, 668.61it/s, Status=SUCCESS]


In [ ]:
### Create data frame and save the data to disk

import pandas as pd
df = pd.DataFrame(data, columns=['kmeans', 'gauss', 'archetype'])
# df.to_csv('gmm-vs-kmeans-benchmark-100percluster.csv')